In [ ]:

import pandas as pd
import numpy as np
import os   

from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

In [ ]:
data_folder = "./data/"
clean_data = os.join(data_folder, "clean_data.csv")

In [ ]:
data = pd.read_csv(clean_data, index_col=0 , header=[0, 1], parse_dates=True, dtype='float64')
data.head() 
